In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [5]:
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state= 0)

In [7]:
# Standardize features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32).view(-1, 1)

In [9]:
# Step 3: Create a PyTorch DataLoader
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

In [10]:
class LogisticRegression(nn.Module):
  def __init__(self, input_size):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(input_size, 1)

  def forward(self, x):
    return torch.sigmoid(self.linear(x))

In [11]:
# Step 5: Define loss function and optimizer
model = LogisticRegression(input_size=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [12]:
# Step 6: Train the model
num_epochs = 100
for epochs in range(num_epochs):
   for inputs, targets in train_loader:
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

In [13]:
# Step 7: Evaluate the model
with torch.no_grad():
  outputs = model(X_test_tensor)
  predictions = (outputs >=0.5).float()
  accuracy = (predictions == Y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.8500000238418579
